In [39]:
from collections import defaultdict
import glob
import numpy as np
import os
import re

import pandas as pd
from tqdm import tqdm

In [40]:
# delta y

In [43]:
from tablebench.core.tabular_dataset import CachedDataset
cache_dir = "../tmp"

from tablebench.notebook_lib import UIDS_LIST, extract_name_from_uid
uids_list = UIDS_LIST


In [45]:
# list of (uid, split, n_split, ymean) tuples
results = []

uids_list = ['physionetICULOSgt47.0']
for uid in uids_list:
    print(f'processing uid {uid}')
    base_dir = os.path.join(cache_dir, uid)  
    if uid == 'physionetICULOSgt47.0':
        name = "physionet"
    else:
        name = extract_name_from_uid(uid)
    dset = CachedDataset(cache_dir=cache_dir, uid=uid, name=name)
    for split in ('train', 'id_test', 'ood_test', 'validation', 'ood_validation'):
        print(f'processing split {split}')
        yvals = list()
        split_files = glob.glob(os.path.join(base_dir, split, '*', '*.csv'))

        for f in tqdm(split_files):
            df = pd.read_csv(f)
            yvals.extend(df[dset.target].values.tolist())
        
        n_split = len(yvals)
        ybar_split = np.array(yvals).mean()
        results.append((uid, split, n_split, ybar_split))
        

processing uid physionetICULOSgt47.0
processing split train


100%|███████████████████████████████████████████████| 274/274 [00:08<00:00, 31.39it/s]


processing split id_test


100%|█████████████████████████████████████████████████| 35/35 [00:01<00:00, 33.56it/s]


processing split ood_test


100%|█████████████████████████████████████████████████| 33/33 [00:01<00:00, 28.03it/s]


processing split validation


100%|█████████████████████████████████████████████████| 35/35 [00:00<00:00, 53.72it/s]


processing split ood_validation


100%|███████████████████████████████████████████████████| 4/4 [00:00<00:00, 64.62it/s]


In [46]:
df = pd.DataFrame(results, columns=['uid', 'split', 'n_split', 'ybar_split'])
df

,uid,split,n_split,ybar_split
0,physionetICULOSgt47.0,train,1122299,0.011850
1,physionetICULOSgt47.0,id_test,140288,0.011997
2,physionetICULOSgt47.0,ood_test,134402,0.075073
3,physionetICULOSgt47.0,validation,140287,0.011833
4,physionetICULOSgt47.0,ood_validation,14934,0.079282


In [47]:
df.pivot(index='uid', columns=['split'])

n_split                                              \
split                 id_test ood_test ood_validation    train validation   
uid                                                                         
physionetICULOSgt47.0  140288   134402          14934  1122299     140287   

                      ybar_split                                               
split                    id_test  ood_test ood_validation    train validation  
uid                                                                            
physionetICULOSgt47.0   0.011997  0.075073       0.079282  0.01185   0.011833

In [48]:
df

,uid,split,n_split,ybar_split
0,physionetICULOSgt47.0,train,1122299,0.011850
1,physionetICULOSgt47.0,id_test,140288,0.011997
2,physionetICULOSgt47.0,ood_test,134402,0.075073
3,physionetICULOSgt47.0,validation,140287,0.011833
4,physionetICULOSgt47.0,ood_validation,14934,0.079282


In [101]:
df.pivot(index='uid', columns=['split']).reset_index().to_csv("dataset_summaries_by_split.csv")

In [49]:
tmp = df.pivot(index='uid', columns=['split'])
ydiff_id_test_ood_test = tmp.loc[:,("ybar_split", "id_test")] - tmp.loc[:,("ybar_split", "ood_test")]
ydiff_id_val_id_test = tmp.loc[:,("ybar_split", "validation")] - tmp.loc[:,("ybar_split", "id_test")]

# (ydiff_id_test_ood_test**2)/abs(ydiff_id_val_id_test)
ydiff_id_test_ood_test**2

# ydiff_id_val_id_test

uid
physionetICULOSgt47.0    0.003979
dtype: float64

# otdd

In [44]:
from otdd.pytorch.distance import DatasetDistance

In [48]:
uid = 'helocdomain_split_varname_ExternalRiskEstimateLowdomain_split_ood_value_0'
base_dir = os.path.join(cache_dir, uid)  
name = extract_name_from_uid(uid)
dset = CachedDataset(cache_dir=cache_dir, uid=uid, name=name)

In [115]:
import torch

def df_to_loader(df, target):
    x = df.drop(columns=target)
    y = df[target]
    x = torch.from_numpy(x.values)
    y = torch.from_numpy(y.values.astype(int))
    tensor_dataset = TensorDataset(x,y)
    loader = torch.utils.data.DataLoader(tensor_dataset, batch_size=64)
    return loader

In [116]:
def read_split_data(base_dir):
    
    split_data = {}
    
    
    # Read the test datasets
    for split in ("ood_test", "id_test"):
        print(split)
        df_list = []
        split_files = glob.glob(os.path.join(base_dir, split, '*', '*.csv'))

        for f in tqdm(split_files):
            df = pd.read_csv(f)
            df_list.append(df)

        split_data[split] = pd.concat(df_list)
    return split_data

In [119]:
results = []

for uid in uids_list:
    print(f"processing {uid}")
    base_dir = os.path.join(cache_dir, uid)  
    
    if uid == 'physionetICULOSgt47.0':
        name = "physionet"
    else:
        name = extract_name_from_uid(uid)
        
    dset = CachedDataset(cache_dir=cache_dir, uid=uid, name=name)
    
    split_data = read_split_data(base_dir)
    
    # Compare 
    assert np.all(split_data["id_test"].dtypes == split_data["ood_test"].dtypes)
    id_loader =  df_to_loader(split_data["id_test"], target=dset.target)
    ood_loader = df_to_loader(split_data["ood_test"], target=dset.target)
#     try:
    dist = DatasetDistance(id_loader, ood_loader,
                           # Note: this fails with 'exact' for some datasets.
                       inner_ot_method = 'gaussian_approx',
                       debiased_loss = True,
                       p = 2, entreg = 1e-1,
                       device='cpu')
    print(f"computing distance for {uid}")
    d = dist.distance(maxsamples = 10000)
    print(f'OTDD(src,tgt)={d}')
    results.append((uid, d))
#     except Exception as e:
#         print(e)
#         continue
    


processing acsfoodstampsdomain_split_varname_DIVISIONdomain_split_ood_value_06
ood_test


100%|███████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 21.26it/s]


id_test


100%|███████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 26.13it/s]


computing distance for acsfoodstampsdomain_split_varname_DIVISIONdomain_split_ood_value_06


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2192 [00:00<?, ?it/s]

  0%|          | 0/2101 [00:00<?, ?it/s]

OTDD(src,tgt)=6554.369140625


In [122]:

otdd_results_df = pd.DataFrame([(x[0], x[1].detach().cpu().numpy().item()) for x in results], columns=['uid', 'otdd'])
otdd_results_df.to_csv("otdd_results", index=False)
otdd_results_df


,uid,otdd
0,acsfoodstampsdomain_split_varname_DIVISIONdoma...,1.415322e+01
1,acsincomedomain_split_varname_DIVISIONdomain_s...,3.068935e+01
2,acspubcovdomain_split_varname_DISdomain_split_...,5.790109e+00
3,acsunemploymentdomain_split_varname_SCHLdomain...,7.469135e+01
4,anesdomain_split_varname_VCF0112domain_split_o...,1.360260e+01
5,brfss_diabetesdomain_split_varname_PRACE1domai...,1.239035e+01
6,brfss_blood_pressuredomain_split_varname_BMI5C...,4.630351e+00
7,diabetes_readmissiondomain_split_varname_admis...,4.249803e+01
8,nhanes_leaddomain_split_varname_INDFMPIRBelowC...,1.297540e+00
9,mimic_extract_los_3domain_split_varname_insura...,5.651536e+10
